加载两个工具类

In [ ]:
# 从 🤗 Diffusers 导入 DiffusionPipeline（把 UNet、VAE、文本编码器、调度器等组件打包到一起的一站式推理管道）。
# 导入 PyTorch。
from diffusers import DiffusionPipeline
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 通过 from_pretrained 从给定仓库/路径加载一个已经配好的扩散模型管道（权重和配置）。
# safety_checker=None 关闭 NSFW 安全过滤器（生成内容将不做额外审查）。
# 这一步会把文本编码器、UNet、VAE、scheduler、tokenizer等子模块都装到 pipeline 里（默认在 CPU 上）。
pipeline = DiffusionPipeline.from_pretrained(
    'lansinuote/diffsion_from_scratch.params', safety_checker=None)

scheduler = pipeline.scheduler # 是往图片中参入噪声的工具类
tokenizer = pipeline.tokenizer

del pipeline # 删除大而全的 pipeline 对象本身以释放内存（权重大、占 RAM/显存）。
             # 已经单独拿出来的 scheduler 与 tokenizer 只是独立引用，不会被删。

# 在 Jupyter/REPL 中，单独一行写多个变量会回显一个元组 (device, scheduler, tokenizer)，便于你看当前三者的值/类型。
# 在普通 .py 文件里，这一行不会打印任何东西（除非 print(...)）。
device, scheduler, tokenizer

KeyboardInterrupt: 

对数据当中的图片和文本分别进行编码

In [ ]:
from datasets import load_dataset
import torchvision

#加载数据集
dataset = load_dataset(path='lansinuote/diffsion_from_scratch', split='train')

#图像增强模块
compose = torchvision.transforms.Compose([
    torchvision.transforms.Resize(
        512, interpolation=torchvision.transforms.InterpolationMode.BILINEAR),
    torchvision.transforms.CenterCrop(512),
    #torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5], [0.5]),
])


def f(data):
    #应用图像增强
    pixel_values = [compose(i) for i in data['image']]
    # map(..., batched=True) 时，data['image'] 是一个批次的列表（长度≤batch_size），对每张图做 compose，得到一堆 [C,512,512] 的张量。

    #文字编码
    input_ids = tokenizer.batch_encode_plus(data['text'],
                                            padding='max_length',
                                            truncation=True,
                                            max_length=77).input_ids # 批量把文本转成 token id：
                                                # 固定 长度 77（CLIP 常用长度），不足则用 pad 补到 77，过长则截断；
                                                # input_ids 是 List[List[int]]，每条样本 77 个 id（末尾很多是 pad id）。

    return {'pixel_values': pixel_values, 'input_ids': input_ids}
    # 返回给 datasets，新列叫 pixel_values 和 input_ids。datasets 会据此推断特征类型与形状。

"""
batched=True：f 一次处理一个批次，更高效；
batch_size=100：每批 100 条数据传给 f；
num_proc=1：单进程映射（设成>1 可多进程并行）；
remove_columns=['image','text']：丢掉原始的 image/text 两列，避免重复存储，最后只保留处理后的两列。
"""
dataset = dataset.map(f,
                      batched=True,
                      batch_size=100,
                      num_proc=1,
                      remove_columns=['image', 'text'])

"""
指定取样本/批次时，把这些列自动转成 PyTorch 张量：
pixel_values：torch.float32，形状 [C,512,512]；
input_ids：torch.int64，形状 [77]。
"""
dataset.set_format(type='torch')

"""
在 Notebook 里，这一行会回显两个值：
dataset 的概况（features、num_rows 等）；
第一条样本的字典：{'pixel_values': tensor(...), 'input_ids': tensor([...])}。
"""
dataset, dataset[0]

Using custom data configuration lansinuote--diffsion_from_scratch-34318fc75271f5a0
Found cached dataset parquet (/root/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--diffsion_from_scratch-34318fc75271f5a0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at /root/.cache/huggingface/datasets/lansinuote___parquet/lansinuote--diffsion_from_scratch-34318fc75271f5a0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-314ecfdae0cf40d9.arrow


(Dataset({
     features: ['pixel_values', 'input_ids'],
     num_rows: 833
 }),
 {'pixel_values': tensor([[[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]],
  
          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]],
  
          [[1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           ...,
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.],
           [1., 1., 1.,  ..., 1., 1., 1.]]]),
  'input_ids': tensor([49406,   320,  3610,   539,   320,  1901,  9528

In [ ]:
#定义loader
def collate_fn(data): # 自定义 打包函数。DataLoader 每次取一个 batch 时，会把该 batch 内的样本列表传进来（data 是一个 list，长度=batch_size；元素是你 dataset[?] 返回的 dict）。
    """
    从这批样本里分别取出图像与文本两列，形成两个 list：
    pixel_values：长度 = batch_size，每个元素形状是 [C, 512, 512] 的张量（float32，范围[-1,1]）。
    input_ids：长度 = batch_size，每个元素形状是 [77] 的张量（int64）。
    """
    pixel_values = [i['pixel_values'] for i in data]
    input_ids = [i['input_ids'] for i in data]

    """
    torch.stack 在新维度上拼接，得到批量张量：
    pixel_values 变为 [B, C, 512, 512]
    input_ids 变为 [B, 77]
    .to(device) 把这两个张量直接搬到设备（'cuda' 或 'cpu'）。
    这样做的好处：每个 batch 产出即在目标设备，后面训练/推理时无需再搬运。
    """
    pixel_values = torch.stack(pixel_values).to(device)
    input_ids = torch.stack(input_ids).to(device)

    # 返回一个批的字典；后续模型前向可直接解包使用。
    return {'pixel_values': pixel_values, 'input_ids': input_ids}


loader = torch.utils.data.DataLoader(dataset,
                                     shuffle=True,
                                     collate_fn=collate_fn,
                                     batch_size=1)

len(loader), next(iter(loader)) # len(loader)：batch 的数量。你的数据集大小之前显示为 833，batch_size=1，因此 len(loader) = 833。
# next(iter(loader))：取第一个 batch（因为 shuffle=True，它是随机的一条）。返回字典：
# batch['pixel_values']：形状 [1, C, 512, 512]，dtype=torch.float32，位于 device
# batch['input_ids']：形状 [1, 77]，dtype=torch.int64，位于 device


(833,
 {'pixel_values': tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            ...,
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.]],
  
           [[1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            ...,
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.]],
  
           [[1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            ...,
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.],
            [1., 1., 1.,  ..., 1., 1., 1.]]]], device='cuda:0'),
  'input_ids': tensor([[49406,   320,  3610,   539,   320,  7651,  4009,   530,  3360,   537,
            

加载模型 准备训练

In [4]:
#加载模型
%run 1.encoder.ipynb
%run 2.vae.ipynb
%run 3.unet.ipynb

#准备训练
encoder.requires_grad_(False)
vae.requires_grad_(False)
unet.requires_grad_(True)

encoder.eval()
vae.eval()
unet.train()

encoder.to(device)
vae.to(device)
unet.to(device)

optimizer = torch.optim.AdamW(unet.parameters(),
                              lr=1e-5,
                              betas=(0.9, 0.999),
                              weight_decay=0.01,
                              eps=1e-8)

criterion = torch.nn.MSELoss()

optimizer, criterion

The config attributes {'scaling_factor': 0.18215} were passed to AutoencoderKL, but are not expected and will be ignored. Please verify your config.json configuration file.


(AdamW (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     eps: 1e-08
     foreach: None
     lr: 1e-05
     maximize: False
     weight_decay: 0.01
 ),
 MSELoss())

计算 Loss

典型的扩散模型（Stable Diffusion 风格）训练一步：固定文本编码器与VAE，只训练UNet，让它在给定t步的噪声水平下预测噪声 ε，用MSE对齐真噪声。

这段代码做了：冻结文本与VAE → 得到条件文本嵌入与潜空间latent → 随机选t把噪声加到latent上形成x_t → 用UNet在(t, 文本条件)下预测噪声ε → 用MSE把预测和真噪声对齐。训练充分后，推理时就能从纯噪声一步步去噪生成潜向量，再经VAE解码回图像。

In [ ]:
"""
定义一个函数做一次前向+计算loss。data 是一个字典，至少包含：
data['input_ids']: 文本token ids，形状 [B, 77]（这里示例B=1）
data['pixel_values']: 原图像张量，形状 [B, 3, 512, 512]
"""
def get_loss(data):
    with torch.no_grad(): # 在这个块里关闭梯度。目的：
                            # 文本编码器 encoder 和 VAE 编码器通常不训练（冻结），
                            # 节省显存和加速前向。
        #文字编码
        #[1, 77] -> [1, 77, 768]
        out_encoder = encoder(data['input_ids'])

        """
        通过VAE把像素空间映射到潜空间（latent）：
        vae.encoder(...) 通常输出高斯分布的参数（如均值μ、logvar），形状内部结构由实现决定；
        vae.sample(...) 做重参数化采样得到潜向量 z，形状为 [B, 4, 64, 64]。
        直觉：512×512 的图片被压到 64×64 的潜空间里，通道数是4。
        """
        #抽取图像特征图
        #[1, 3, 512, 512] -> [1, 4, 64, 64]
        out_vae = vae.encoder(data['pixel_values'])
        out_vae = vae.sample(out_vae)

        #0.18215 = vae.config.scaling_factor
        out_vae = out_vae * 0.18215 # 关键细节：Stable Diffusion 使用固定缩放因子 0.18215，确保潜空间的数值范围与训练时一致。
                                    # 训练和推理两边都要保持这个缩放一致，否则噪声强度/信噪比会错位，导致训练/推理不匹配。

    """
    采样与 latent 同形状的标准高斯噪声 ε：
    形状 [B, 4, 64, 64]
    这就是训练的监督信号（目标），后面用MSE让UNet去回归它。
    """
    #随机数,unet的计算目标
    noise = torch.randn_like(out_vae)

    #往特征图中添加噪声
    #1000 = scheduler.num_train_timesteps
    #1 = batch size
    noise_step = torch.randint(0, 1000, (1, )).long().to(device)
    """
    随机选一个时间步 t：
    值域 [0, 999]（共1000个训练步，取决于scheduler配置）
    形状是 [1]（批量是1）。
    如果以后 B>1，通常写成 torch.randint(0, 1000, (B,))，或 [1] 再广播也行，但很多实现要求 [B]，更稳妥。
    """
    out_vae_noise = scheduler.add_noise(out_vae, noise, noise_step)
    """
    把噪声按扩散公式加到latent上，得到 x_t：
    数学：x_t = sqrt(ᾱ_t) * x_0 + sqrt(1 - ᾱ_t) * ε
    其中 x_0 就是 out_vae（缩放后的latent），ε 是上面采的噪声，ᾱ_t 是scheduler（DDPM/DDIM等）给定的累计噪声计划。
    输出形状仍是 [B, 4, 64, 64]。
    scheduler.add_noise 内部会查到 t 对应的 sqrt_alpha_cumprod[t] 与 sqrt_one_minus_alpha_cumprod[t] 并做线性组合。
    """

    #根据文字信息,把特征图中的噪声计算出来
    out_unet = unet(out_vae=out_vae_noise,
                    out_encoder=out_encoder,
                    time=noise_step)
    """
    把带噪潜向量 x_t、文本条件、时间步 t 一起喂入UNet：
    out_vae_noise: [B, 4, 64, 64]
    out_encoder: [B, 77, 768]（做Cross-Attn条件）
    time: [B]或[1]（你的UNet里一般会先把t过时间嵌入，如正弦位置编码+MLP → 1280维等）
    UNet输出通常是对噪声 ε 的预测：
    out_unet 形状 [B, 4, 64, 64]
    """

    #计算mse loss
    #[1, 4, 64, 64],[1, 4, 64, 64]
    return criterion(out_unet, noise)


# get_loss({
#     'input_ids': torch.ones(1, 77, device=device).long(),
#     'pixel_values': torch.randn(1, 3, 512, 512, device=device)
# })

一共训练400个epoch，每四个批次我们进行一次参数的调整 

它的作用
torch.nn.utils.clip_grad_norm_(unet.parameters(), 1.0) =
把 unet 所有参数的**梯度向量的整体长度（L2 范数）**限制在 1.0 以内，防止梯度过大（梯度爆炸）。

为什么要这么做
梯度太大 → 一步更新跳得过猛 → 训练不稳定甚至发散。
裁剪后：方向不变，长度变小，更新更稳。

它具体干了三步

- 把所有参数的梯度拼成一个“大向量”，算它的 L2 范数 total_norm。
- 如果 total_norm ≤ 1.0：啥也不做。
-  total_norm > 1.0：把所有梯度同时乘以 1.0 / total_norm（统一缩放），让整体范数正好变成 1.0。

超简例子
假设两个参数的梯度是 g1=3、g2=4，整体范数 sqrt(3^2+4^2)=5 > 1。
缩放系数 = 1/5 = 0.2，所以新梯度变为 g1=0.6、g2=0.8。
→ 方向相同（比例没变），只是整体“长度”从 5 缩到 1。

放在你这段代码里的含义
你每累计 4 次 loss.backward() 才 optimizer.step()，这行就在 step 之前把累计后的总梯度裁剪到 ≤ 1.0，保证这次更新不会过猛。

两点补充
- 这是按范数裁剪（clip_grad_norm_），不同于按数值逐元素截断（clip_grad_value_）。
- 函数会原地修改梯度，并返回裁剪前的范数（可用来打印监控）。

In [6]:
def train():
    loss_sum = 0
    for epoch in range(400):
        for i, data in enumerate(loader):
            loss = get_loss(data) / 4
            loss.backward()
            loss_sum += loss.item()

            if (epoch * len(loader) + i) % 4 == 0:
                torch.nn.utils.clip_grad_norm_(unet.parameters(), 1.0)
                optimizer.step()
                optimizer.zero_grad()

        if epoch % 10 == 0:
            print(epoch, loss_sum)
            loss_sum = 0

    #torch.save(unet.to('cpu'), 'saves/unet.model')


train()

0 11.7118999005761
10 105.27776907754014
20 101.45478522218764
30 97.96161541804031
40 95.7652038520173
50 92.64628775657911
60 91.62508884524868
70 88.90302349776903
80 84.6358380591555
90 82.70271758512536
100 81.53195204613439
110 76.3927595877758
120 74.14106083381193
130 71.42537906522921
140 69.16221529991162
150 65.47076485656726
160 62.1360088881047
170 60.89056803673884
180 57.985315461344726
190 54.73302427918679
200 50.69724302080431
210 48.59712202517403
220 46.407517315681616
230 44.99496047659704
240 44.07751854383969
250 39.62402399040002
260 37.051896732489695
270 36.89249631060375
280 35.71413582353853
290 33.45783720578038
300 33.08240255239798
310 30.282505852694158
320 29.86848702972202
330 29.363934024146147
340 29.187604612583527
350 27.543819716789585
360 26.130621485815936
370 25.465440133120865
380 25.48384229660587
390 24.789676978944044


In [7]:
from transformers import PreTrainedModel, PretrainedConfig


#包装类
class Model(PreTrainedModel):
    config_class = PretrainedConfig

    def __init__(self, config):
        super().__init__(config)
        self.unet = unet.to('cpu')

#保存到hub
Model(PretrainedConfig()).push_to_hub(
    repo_id='lansinuote/diffsion_from_scratch.unet',
    use_auth_token=open('/root/hub_token.txt').read().strip())

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lansinuote/diffsion_from_scratch.unet/commit/32f5e4163edb6d1a3fa1d8265ad2cdf0406cb425', commit_message='Upload model', commit_description='', oid='32f5e4163edb6d1a3fa1d8265ad2cdf0406cb425', pr_url=None, pr_revision=None, pr_num=None)